<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Cutmix_and_Mixup_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CutMix and Mixup Augmentation Startegies
**CutMix**:
In CutMix augmentation we cut and paste random patches between the training images. The ground truth labels are mixed in proportion to the area of patches in the images. CutMix increases localization ability by making the model to focus on less discriminative parts of the object being classified and hence is also well suited for tasks like object detection.

Import the necessary libraries

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

First, take the dataset and unbatch it and zip them to create a new dataset.

In [ ]:
train1 = train_df.shuffle(5400).unbatch()
train2 = train_df.shuffle(5400).unbatch()
mixed_data = tf.data.Dataset.zip((train1, train2))

Then, Define the function with cutmix logic.
In below function we have passed the two datasets. We create the coordinates for bounding boxes.


1.   For X and Y coordinates we take the uniform distribution.
2.   The, we create a lambda value from beta distribution for height and weidth of BB_BOXES.
3.   Take square root of 1 - lambda and multiply it with image size.
4.   For bounding boxes clip the output values between 0 and Image size.
5.   Check if target height and target width is <= 0, if yes the keep the values equal to 1.
6. The, pass these bounding boxes values to tf function crop_to_bounding_box to get the patch then add the padding to that patch.
7.  The, repeat the same process the get the patch from image 2.
8.  For final image: subtract the pad_image from image 1 and add the padded_image patch from image 2.
9.  For new_label: Follow below new label line.



In [ ]:
def Cutmix(data1, data2):

  (image1, label1), (image2, label2) = data1, data2
  IMG_SIZE = 200

  cut_x = tf.cast(tfp.distributions.Uniform(0,200).sample(1)[0], dtype=tf.int32)
  cut_y = tf.cast(tfp.distributions.Uniform(0,200).sample(1)[0], dtype=tf.int32)

  lamda = tfp.distributions.Beta(0.2,0.2)
  lamda = lamda.sample(1)[0]

  cut_h = tf.cast(IMG_SIZE * (tf.math.sqrt(1-lamda)), dtype=tf.int32)
  cut_w = tf.cast(IMG_SIZE * (tf.math.sqrt(1-lamda)), dtype=tf.int32)

  bbx1 = tf.clip_by_value(cut_x - cut_h//2, 0, IMG_SIZE)
  bby1 = tf.clip_by_value(cut_y - cut_w//2, 0, IMG_SIZE)
  bbx2 = tf.clip_by_value(cut_x + cut_h//2, 0, IMG_SIZE)
  bby2 = tf.clip_by_value(cut_y + cut_w//2, 0, IMG_SIZE)

  target_height = bbx2 - bbx1
  if (target_height == 0):
    target_height = 1
  target_width = bby2 - bby1
  if (target_width == 0):
    target_width = 1

  crop1 = tf.image.crop_to_bounding_box(image1, bbx1, bby1, target_height, target_width)
  img_pad1 = tf.image.pad_to_bounding_box(crop1, bbx1,bby1, IMG_SIZE, IMG_SIZE)

  crop2 = tf.image.crop_to_bounding_box(image2, bbx1,bby1, target_height, target_width)
  img_pad2 = tf.image.pad_to_bounding_box(crop2, bbx1,bby1, IMG_SIZE, IMG_SIZE)

  final_image = image1 - img_pad1 +img_pad2
  lamda = tf.cast(1-(target_width*target_height)/(IMG_SIZE*IMG_SIZE), dtype=tf.float32)
  new_label = lamda*tf.cast(label1, dtype=tf.float32) + (1-lamda)*tf.cast(label2, dtype=tf.float32)

  return final_image, new_label

map the above function to the dataset. Shuffle the date.

In [ ]:
mixup_data = mixed_data.shuffle(5000).map(Cutmix)

Batch the above dataset.

In [ ]:
mixup_data = mixup_data.batch(32)

# Mixup Augmentation Method
Mixup is a data augmentation technique that involves blending pairs of samples and their corresponding labels to create new synthetic training examples.

In [8]:
def mixup(data1, data2):
  # unpack the datasets
  (image1, label1), (image2, label2) = data1, data2
  # create a beta distribution value called lamda
  lamda = tfp.distributions.Beta(0.2,0.2).sample(1)[0]

  new_image = lamda*image1 + (1-lamda)*image2
  new_label = lamda*label1 + (1-lamda)*label2

  return new_image, new_label